<center> <h1 style="background-color:seagreen; color:white" >Load Data</h1> 

In [ ]:
# Install PySpark
!pip install pyspark > /dev/null

In [ ]:
# Req libraries
import os
import numpy as np
import pandas as pd

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [ ]:
# Spark Session
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('fashion-recommendations').getOrCreate()

In [ ]:
# Load Data
transaction = spark.read.option('header','true').csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

<center> <h1 style="background-color:seagreen; color:white" >Feature Engineering</h1> 

In [ ]:
# filter only 2020
sales =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
sales = sales.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
sales = sales.withColumn('year', year(col('date')))
sales = sales.withColumn('month', month(col('date')))

# Let's filter the data to start with
sales = sales[sales['year'] == 2020]
sales = sales[sales['month'] == 1]

transaction.unpersist()

# Prepare the dataset
sales = sales.groupby('customer_id', 'article_id').count()
sales.show(5)

<center> <h1 style="background-color:seagreen; color:white" >Alternative Least Sqaure</h1>

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

<center> <h1 style="background-color:DarkKhaki; color:white" >Converting String to Index</h1>

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(sales.columns)-set(['count'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(sales).transform(sales)
transformed.show(5)

<center> <h1 style="background-color:DarkKhaki; color:white" >Creating ALS model and fitting data</h1>

In [ ]:
(training,test)=transformed.randomSplit([0.8, 0.2])

als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

In [ ]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="count",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show(5)